# Downloading ICON Forecast Data via Python API
This notebook guides you through the download process of a forecast file from the ICON numerical weather model using the `ogd_api` module. The `ogd_api` module is part of the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — an in-house developed toolkit designed to simplify working with numerical weather model data.

The data you will access is provided by MeteoSwiss as part of Switzerland’s Open Government Data (OGD) initiative.

---

## 🔍 **What You’ll Do in This Notebook**

⬇️  **Download Forecast Files**  
    Retrieve and save the requested ICON forecast files (e.g. soil temperature) to disk for offline storage using `ogd_api.download_from_ogd`.

✅  **(Optional) Verify Data Integrity**  
    Ensure the downloaded files are complete and uncorrupted by checking their SHA-256 hashes.

---

##  ⬇️ Downloading Forecast Files
In this first part, we retrieve soil temperature forecast data from the ICON numerical weather model. To download this data, we use the `ogd_api` module from the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library which provides a convenient interface for accessing numerical weather forecasts via the [STAC (SpatioTemporal Asset Catalog) API](https://data.geo.admin.ch/api/stac/static/spec/v1/apitransactional.html#tag/Data/operation/getAsset).

#### 📁  Browsing the STAC Catalog (Optional)

If you'd like to explore the ICON forecast datasets interactively before downloading, you can browse them directly in the STAC catalog:

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH1-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en)

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH2-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch2?.language=en)


Below is a screenshot of the ICON-CH2-EPS collection as seen in the STAC browser interface.

![browser-ch2-resized.PNG](./images/browser-ch2-resized.PNG)

### Creating a Request
To retrieve ICON-CH1-EPS soil temperature (`T_SO`), we first define an API request using the `ogd_api.Request` class.
The request targets lead time zero - the forecast's initialization time.

>⏰ **Forecast Availability**: Forecast data will typically be available a couple of hours after the reference time — due to the model runtime and subsequent upload time. The data remains accessible for 24 hours after upload.

In [13]:
from datetime import datetime, timezone
from meteodatalab import ogd_api

today_midnight_utc = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)

req = ogd_api.Request(
    collection="ogd-forecasting-icon-ch1",
    variable="T_SO",
    reference_datetime=today_midnight_utc,
    perturbed=False,
    horizon="P0DT0H",
)

Each argument in the request serves the following purpose:

| Argument             | Description |
|----------------------|-------------|
| `collection`         | Forecast collection to use (e.g., `ogd-forecasting-icon-ch1`). |
| `variable`           | Meteorological variable of interest (`T_SO` = soil temperature). |
| `reference_datetime` | Initialization time of the forecast in **UTC**, provided as either:<br>- [datetime.datetime](https://docs.python.org/3/library/datetime.html#datetime-objects) object (e.g.,<br> &nbsp; `datetime.datetime(2025, 5, 22, 9, 0, 0, tzinfo=datetime.timezone.utc)`) <br>- [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Combined_date_and_time_representations) date string (e.g., `"2025-05-22T09:00:00Z"`)|
| `perturbed`          | If `True`, retrieves ensemble forecast members; if `False`, returns the deterministic forecast. |
| `horizon`            | Forecast lead time, provided as either:<br>– [datetime.timedelta](https://docs.python.org/3/library/datetime.html#timedelta-objects) object (e.g., `datetime.timedelta(hours=0)`) <br>– [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Durations) duration string (e.g., `"P0DT0H"`)|

### Downloading Data
We now send our request to the API and download the resulting dataset using the `download_from_ogd()` function. The function expects a request and a path to a directory, where the data will be stored. In this example the forecast file is stored in the folder `forecast_files` within this directory.

> 💡 **Hint**: If you get an error message containing "HTTPError: 403 Client Error: Forbidden for url", you may be trying to retrieve data older than 24h hours! Please adjust your requests.

In [14]:
from pathlib import Path

# Define the target directory for saving the forecast files
target_dir = Path.cwd() / "forecast_files"

# Download the forecast files
ogd_api.download_from_ogd(req, target_dir)

# List all downloaded files in the target directory
print("\n Downloaded files:")
for file in sorted(target_dir.iterdir()):
    print(f" - {file.name}")


 Downloaded files:
 - horizontal_constants_icon-ch1-eps.grib2
 - horizontal_constants_icon-ch1-eps.sha256
 - icon-ch1-eps-202504290000-0-t_so-ctrl.grib2
 - icon-ch1-eps-202504290000-0-t_so-ctrl.sha256
 - vertical_constants_icon-ch1-eps.grib2
 - vertical_constants_icon-ch1-eps.sha256


After downloading, you should find the following files inside the `forecast_files/` directory:

- `horizontal_constants_icon-ch1-eps.grib2`
- `horizontal_constants_icon-ch1-eps.sha256`
- `icon-ch1-eps-<today's-datetime>-0-t_so-ctrl.grib2`
- `icon-ch1-eps-<today's-datetime>-0-t_so-ctrl.sha256`
- `vertical_constants_icon-ch1-eps.grib2`
- `vertical_constants_icon-ch1-eps.sha256`

### 📦 Content of the 'forecast_files' Directory

- **Main forecast file** (`icon-ch1-eps-<datetime>-0-t_so-ctrl.grib2`)  
  Contains the forecast data, e.g., soil temperature at lead time +0h.

- **Horizontal constants** (`horizontal_constants_icon-ch1-eps.grib2`)  
  Contains the longitude and latitude for every grid point.

- **Vertical constants** (`vertical_constants_icon-ch1-eps.grib2`)  
  Contains the height of the vertical half levels.

- **SHA-256 checksum files** (`*.sha256`)  
  Used to verify the integrity of each corresponding GRIB2 file.

> **Note:**  
> Forecast GRIB files like `icon-ch1-eps-*.grib2` do **not** include geographic grid information (longitude, latitude, or height) internally.  
> Therefore, the horizontal and vertical constants are provided as separate files to fully describe the forecast grid.  
>  
> Learn more about the data structure [here](https://opendatadocs.meteoswiss.ch/e-forecast-data/e2-e3-numerical-weather-forecasting-model#3d-grid-structure-and-representation).

## ✅ (Optional) Verify Data Integrity

In the second part we ensure that the downloaded file is not corrupted, compute its SHA-256 hash and verify it against the checksum provided in the SHA256 file.

Steps:

1. Generate the SHA-256 checksum of the downloaded file:

In [15]:
!sha256sum forecast_files/horizontal_constants_icon-ch1-eps.grib2

9a7e1243398e5d66311ee563539f1252e49ebad499f448c924b756b4ebd0aeee  forecast_files/horizontal_constants_icon-ch1-eps.grib2


2. Retrieve the checksum from the SHA256 file using the following command:

In [16]:
cat forecast_files/horizontal_constants_icon-ch1-eps.sha256

9a7e1243398e5d66311ee563539f1252e49ebad499f448c924b756b4ebd0aeee

3. Compare the two hash values. If they match, your forecast data file is safe to use.